# Assesment Problems

## Problem 1 — Extending the Lady Tasting Tea

### Plan

**Goal.** Under the null hypothesis (no skill), estimate the probability a participant correctly identifies *all* cups by chance in:
- the classic **8-cup** design (4 tea-first, 4 milk-first), and
- the **12-cup** extension (8 tea-first, 4 milk-first).

**Approach.**
1) The participant must choose exactly the `n_milk` “milk-first” indices; the rest are tea-first.  
2) Under random guessing with the correct class sizes, “all correct” happens only if their chosen subset equals the true subset.  
3) The exact probability is therefore:  
   \[
   \Pr(\text{all correct}) = \frac{1}{\binom{n_\text{total}}{n_\text{milk}}}.
   \]
4) We also verify by simulation.  
5) Briefly discuss how the extended design affects evidence thresholds (p-values) and options for rejection regions.